### UNIÓN DE LOS EVENTOS, CON SU UBICACION Y LOCALIZACIÓN EXACTA (latitud y longitud)

In [1]:
import requests as req
import re
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../src')

- Llega la parte en la que unimos los datos de ubicacion en el dataframe de ENTRADAS

In [2]:
df_event = pd.read_csv('../data/ENTRADAS.csv')

In [3]:
df_event.head()

,nombre,ubicacion,precio,enlace,año,mes,dia,hora
0,El Rey León,Teatro Lope de Vega,49.00 €,https://www.entradas.com/event/el-rey-leon-el-...,2023,11,30,19:00:00
1,El Rey León,Teatro Lope de Vega,45.00 €,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,1,17:00:00
2,El Rey León,Teatro Lope de Vega,45.00 €,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,1,21:00:00
3,El Rey León,Teatro Lope de Vega,67.00 €,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,2,17:00:00
4,El Rey León,Teatro Lope de Vega,45.00 €,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,2,21:00:00


In [4]:
df_event = df_event.rename(columns={'ubicacion': 'local'})

In [5]:
df_event.head()

,nombre,local,precio,enlace,año,mes,dia,hora
0,El Rey León,Teatro Lope de Vega,49.00 €,https://www.entradas.com/event/el-rey-leon-el-...,2023,11,30,19:00:00
1,El Rey León,Teatro Lope de Vega,45.00 €,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,1,17:00:00
2,El Rey León,Teatro Lope de Vega,45.00 €,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,1,21:00:00
3,El Rey León,Teatro Lope de Vega,67.00 €,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,2,17:00:00
4,El Rey León,Teatro Lope de Vega,45.00 €,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,2,21:00:00


In [6]:
df_locales = pd.read_csv('../data/df_locales.csv')

In [7]:
df_locales.head()

,local,ubicacion
0,Teatro Lope de Vega,"C.Gran Vía, 57, 28013 Madrid"
1,Bajo la Gran Carpa,NaN
2,Nuevo Teatro Alcalá,"C. de Jorge Juan, 62, 28009 Madrid"
3,Teatro Rialto,"C.Gran Vía, 54, 28013 Madrid"
4,WiZink Center,"Av. de Felipe II, s/n, 28009 Madrid"


In [8]:
df_locales["Lat&Lon"] = "equis"

In [10]:
df_locales.ubicacion[0]

'C.Gran Vía, 57, 28013 Madrid'

In [11]:
x = 'C.Gran Vía, 57, 28013 Madrid'

Importamos la librería, con Nominatim.


La librería Geopy proporciona herramientas para la geocodificación (conversión de direcciones a coordenadas geográficas) y la inversa, la inversa (obtención de direcciones a partir de coordenadas geográficas). En este caso, se menciona el módulo Nominatim, que utiliza el servicio de geocodificación gratuito de OpenStreetMap.

In [12]:
from geopy.geocoders import Nominatim

def obtener_coordenadas(direccion):
    """
    Obtiene las coordenadas (latitud y longitud) a partir de una dirección.

    Parameters:
    - direccion (str): Dirección que incluye la calle y el número.

    Returns:
    - Tuple[float, float]: Tupla con la latitud y longitud.
    """
    geolocalizador = Nominatim(user_agent="mi_aplicacion_geopy")
    ubicacion = geolocalizador.geocode(direccion)

    if ubicacion:
        latitud, longitud = ubicacion.latitude, ubicacion.longitude
        return latitud, longitud
    else:
        print(f"No se pudo obtener la ubicación para la dirección: {direccion}")


In [13]:
obtener_coordenadas(x)

(40.4217903, -3.7086754)

In [14]:
# Pongo en la función un filtro, ya que a la primera ronda a veces no sale alguna de las ubicaciones. 
# De esta forma, insertando un bucle while (con un máximo de 3 vueltas), intenta recoger cada ubicación hasta 3 veces. 
# Si aun así no lo consigue, entonces pido que me ponga un mensaje, para saber cuáles han sido las ubicaciones no encontradas. 


from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
import time

def obtener_coordenadas(direccion):
    """
    Obtiene las coordenadas (latitud y longitud) a partir de una dirección.

    Parameters:
    - direccion (str): Dirección que incluye la calle y el número.

    Returns:
    - Tuple[float, float]: Tupla con la latitud y longitud.
    """
    geolocalizador = Nominatim(user_agent="mi_aplicacion_geopy")

    intentos = 0
    while intentos < 3:
        try:
            ubicacion = geolocalizador.geocode(direccion)
            if ubicacion:
                latitud, longitud = ubicacion.latitude, ubicacion.longitude
                return latitud, longitud
            else:
                print(f"No se pudo obtener la ubicación para la dirección: {direccion}")
                return None
        except (GeocoderTimedOut, GeocoderUnavailable) as e:
            print(f"Error: {e}. Intentando de nuevo...")
            time.sleep(2)
            intentos += 1

    print(f"No se pudo obtener la ubicación para la dirección: {direccion}")
    return None

In [15]:
df_locales["Lat&Lon"] = df_locales["ubicacion"].apply(obtener_coordenadas)

Error: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=C.+de+Jorge+Juan%2C+62%2C+28009+Madrid&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")). Intentando de nuevo...
Error: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=C.+de+Jorge+Juan%2C+62%2C+28009+Madrid&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")). Intentando de nuevo...
Error: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=C.+de+Jorge+Juan%2C+62%2C+28009+Madrid&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")). Intentando de nuevo...
No se pudo

In [32]:
df_locales.head(10)

,local,ubicacion,Lat&Lon
0,Teatro Lope de Vega,"C.Gran Vía, 57, 28013 Madrid","(40.4217903, -3.7086754)"
1,Bajo la Gran Carpa,NaN,"(46.3144754, 11.0480288)"
2,Nuevo Teatro Alcalá,"C. de Jorge Juan, 62, 28009 Madrid",None
3,Teatro Rialto,"C.Gran Vía, 54, 28013 Madrid","(40.421301, -3.7072389)"
4,WiZink Center,"Av. de Felipe II, s/n, 28009 Madrid","(40.423930600000006, -3.6742277376233554)"
5,Sala La Riviera,"P.º Bajo de la Virgen del Puerto, S/N, 28005 M...",None
6,Sala La Paqui,"C. de Barceló, 11, Local B, 28004 Madrid",None
7,Palacio Municipal de IFEMA MADRID,"Av. de la Capital de España, 5, 28042 Madrid","(40.461128, -3.6153329)"
8,The World Of Hans Zimmer: A New Dimension,NaN,"(46.3144754, 11.0480288)"
9,Teatro La Latina,"Pl. de la Cebada, 2, 28005 Madrid","(40.4115711, -3.708772971526286)"


In [26]:
df_locales.to_csv('../data/ubica_evento.csv', index=False, encoding='utf-8')

In [2]:
df_locales = pd.read_csv('../data/ubica_evento.csv')

In [4]:
df_event = pd.read_csv('../data/ENTRADAS.csv')

In [7]:
df_locales.head()

,local,ubicacion,Lat&Lon
0,Teatro Lope de Vega,"C.Gran Vía, 57, 28013 Madrid","(40.4217903, -3.7086754)"
1,Bajo la Gran Carpa,NaN,"(46.3144754, 11.0480288)"
2,Nuevo Teatro Alcalá,"C. de Jorge Juan, 62, 28009 Madrid",NaN
3,Teatro Rialto,"C.Gran Vía, 54, 28013 Madrid","(40.421301, -3.7072389)"
4,WiZink Center,"Av. de Felipe II, s/n, 28009 Madrid","(40.423930600000006, -3.6742277376233554)"


In [13]:
df_event = df_event.rename(columns={'ubicacion': 'local'})

In [14]:
df_event.head()

,nombre,local,precio,enlace,año,mes,dia,hora
0,El Rey León,Teatro Lope de Vega,49.00 €,https://www.entradas.com/event/el-rey-leon-el-...,2023,11,30,19:00:00
1,El Rey León,Teatro Lope de Vega,45.00 €,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,1,17:00:00
2,El Rey León,Teatro Lope de Vega,45.00 €,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,1,21:00:00
3,El Rey León,Teatro Lope de Vega,67.00 €,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,2,17:00:00
4,El Rey León,Teatro Lope de Vega,45.00 €,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,2,21:00:00


In [15]:
# Realizo el merge de los DataFrames

merged_data = pd.merge(df_event, df_locales[['local', 'ubicacion', 'Lat&Lon']], on='local', how='left')

In [16]:
merged_data.head()

,nombre,local,precio,enlace,año,mes,dia,hora,ubicacion,Lat&Lon
0,El Rey León,Teatro Lope de Vega,49.00 €,https://www.entradas.com/event/el-rey-leon-el-...,2023,11,30,19:00:00,"C.Gran Vía, 57, 28013 Madrid","(40.4217903, -3.7086754)"
1,El Rey León,Teatro Lope de Vega,45.00 €,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,1,17:00:00,"C.Gran Vía, 57, 28013 Madrid","(40.4217903, -3.7086754)"
2,El Rey León,Teatro Lope de Vega,45.00 €,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,1,21:00:00,"C.Gran Vía, 57, 28013 Madrid","(40.4217903, -3.7086754)"
3,El Rey León,Teatro Lope de Vega,67.00 €,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,2,17:00:00,"C.Gran Vía, 57, 28013 Madrid","(40.4217903, -3.7086754)"
4,El Rey León,Teatro Lope de Vega,45.00 €,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,2,21:00:00,"C.Gran Vía, 57, 28013 Madrid","(40.4217903, -3.7086754)"


In [17]:
# Convertir la columna de precios a formato numérico
merged_data['precio'] = pd.to_numeric(merged_data['precio'].str.replace(' €', '').replace(',', ''), errors='coerce')

In [18]:
merged_data['hora'] = merged_data['hora'].str.slice(0, 5)

In [19]:
merged_data.head()

,nombre,local,precio,enlace,año,mes,dia,hora,ubicacion,Lat&Lon
0,El Rey León,Teatro Lope de Vega,49.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,11,30,19:00,"C.Gran Vía, 57, 28013 Madrid","(40.4217903, -3.7086754)"
1,El Rey León,Teatro Lope de Vega,45.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,1,17:00,"C.Gran Vía, 57, 28013 Madrid","(40.4217903, -3.7086754)"
2,El Rey León,Teatro Lope de Vega,45.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,1,21:00,"C.Gran Vía, 57, 28013 Madrid","(40.4217903, -3.7086754)"
3,El Rey León,Teatro Lope de Vega,67.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,2,17:00,"C.Gran Vía, 57, 28013 Madrid","(40.4217903, -3.7086754)"
4,El Rey León,Teatro Lope de Vega,45.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,2,21:00,"C.Gran Vía, 57, 28013 Madrid","(40.4217903, -3.7086754)"


In [20]:
evento = merged_data.copy()

In [21]:
evento['Lat&Lon'] = evento['Lat&Lon'].astype(str)


lat_lon_pattern = r'\(([^,]+), ([^)]+)\)'
evento[['lat', 'long']] = evento['Lat&Lon'].str.extract(lat_lon_pattern)


In [25]:

# Convierte las nuevas columnas a tipos numéricos si es necesario
evento[['lat', 'long']] = evento[['lat', 'long']].apply(pd.to_numeric, errors='coerce')

# Elimina la columna "Lat&Lon"
evento = evento.drop('Lat&Lon', axis=1)

In [26]:
evento.head()

,nombre,local,precio,enlace,año,mes,dia,hora,ubicacion,lat,long
0,El Rey León,Teatro Lope de Vega,49.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,11,30,19:00,"C.Gran Vía, 57, 28013 Madrid",40.42179,-3.708675
1,El Rey León,Teatro Lope de Vega,45.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,1,17:00,"C.Gran Vía, 57, 28013 Madrid",40.42179,-3.708675
2,El Rey León,Teatro Lope de Vega,45.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,1,21:00,"C.Gran Vía, 57, 28013 Madrid",40.42179,-3.708675
3,El Rey León,Teatro Lope de Vega,67.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,2,17:00,"C.Gran Vía, 57, 28013 Madrid",40.42179,-3.708675
4,El Rey León,Teatro Lope de Vega,45.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,2,21:00,"C.Gran Vía, 57, 28013 Madrid",40.42179,-3.708675


In [27]:
evento['ubicacion'] = evento['ubicacion'].fillna('No disponible')

In [28]:
evento['lat'] = evento['lat'].fillna('No disponible')
evento.loc[evento['ubicacion'] == 'No disponible', 'lat'] = 'No disponible'

In [29]:
evento['long'] = evento['long'].fillna('No disponible')
evento.loc[evento['ubicacion'] == 'No disponible', 'long'] = 'No disponible'

In [30]:
evento.head()

,nombre,local,precio,enlace,año,mes,dia,hora,ubicacion,lat,long
0,El Rey León,Teatro Lope de Vega,49.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,11,30,19:00,"C.Gran Vía, 57, 28013 Madrid",40.42179,-3.708675
1,El Rey León,Teatro Lope de Vega,45.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,1,17:00,"C.Gran Vía, 57, 28013 Madrid",40.42179,-3.708675
2,El Rey León,Teatro Lope de Vega,45.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,1,21:00,"C.Gran Vía, 57, 28013 Madrid",40.42179,-3.708675
3,El Rey León,Teatro Lope de Vega,67.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,2,17:00,"C.Gran Vía, 57, 28013 Madrid",40.42179,-3.708675
4,El Rey León,Teatro Lope de Vega,45.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,2,21:00,"C.Gran Vía, 57, 28013 Madrid",40.42179,-3.708675


In [31]:
#Creo un dataframe que combina los eventos, con su ubicación (calle), y latitud y longitud. 

evento.to_csv('../data/eventos_def.csv', index=False, encoding='utf-8')